In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, current_timestamp, regexp_replace, count

In [0]:
spark = SparkSession.builder\
    .appName("gold_layer")\
    .getOrCreate()

In [0]:
SILVER_PATH = "/mnt/breweries/silver/breweries.parquet"
GOLD_PATH = "/mnt/breweries/gold/"

In [0]:
def gold_layer():
    if not dbutils.fs.ls(SILVER_PATH):
        return 'No data found in Silver layer'

    df_silver = spark.read.parquet(SILVER_PATH)
    df_gold = df_silver.groupBy("state", "brewery_type").agg(count("id").alias("brewery_count"))

    df_gold = df_gold.orderBy(["state", "brewery_type", "brewery_count"], ascending=[True, True, False])

    # Salvando csv
    df_gold.write.mode('overwrite').option("header", "true").csv(GOLD_PATH)

    return f'Gold layer updated with aggregated data, output_path: {GOLD_PATH}'

In [0]:
print('RUNNING GOLD LAYER...')
print(gold_layer())
print('GOLD LAYER FINISH WITH SUCCESS!')